# OCR Analysis

In [3]:
from PIL import Image
import PIL
import numpy as np
import os

## EasyOCR

In [44]:
import easyocr
reader = easyocr.Reader(['en','es'])

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Now we will try doing infernce on the books we have

In [57]:
# img = np.array(Image.open('./crops/bbox_0.jpg').rotate(90,PIL.Image.NEAREST, expand = 1))
img = Image.open('./crops/bbox_13.jpg')
result = reader.readtext(img, rotation_info=[90])

for (bbox, text, prob) in result:
    print(f'Text: {text}, Probability: {prob}')

Text: 1, Probability: 0.9871954987295481
Text: 1, Probability: 0.970909126936764
Text: 31, Probability: 0.5365686961993075
Text: FIFTH, Probability: 0.7934720202354075
Text: M, Probability: 0.9991555805199006
Text: SENGE, Probability: 0.997829229653294
Text: DISCIPLINE, Probability: 0.9958764014662328
Text: THE ART, Probability: 0.6998610951943425
Text: 8, Probability: 0.6737795794895227
Text: 1, Probability: 0.901520527661205
Text: =, Probability: 0.899389225867143
Text: THE, Probability: 0.9671545561333088
Text: 1, Probability: 0.8574332786865035
Text: 1, Probability: 0.9236438611158668


In [59]:
for file in os.listdir("./crops"):
    print('book')
    img = Image.open(os.path.join('./crops',file))
    for angle in [0,90,-90]:
        result = reader.readtext(img, rotation_info=[angle])

        for (bbox, text, prob) in result:
            print(f'Text: {text}, Probability: {prob}')
        print('-'*20)

book


c:\Users\gerar\OneDrive\Desktop\bookAnalysis\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Text: 4, Probability: 0.6901684203249978
Text: 1, Probability: 0.31540510401329414
Text: {, Probability: 0.452091057570275
Text: 6, Probability: 0.05556251746803387
Text: f, Probability: 0.4787417144996233
Text: ;, Probability: 0.05913410234335026
Text: RANDOM, Probability: 0.9976557258959531
Text: HOUSE, Probability: 0.9987140698819087
--------------------
Text: THE, Probability: 0.9998514071913761
Text: BLACK, Probability: 0.9958218245440462
Text: SWAN, Probability: 0.9052678942680359
Text: Nassmm, Probability: 0.4139407904126125
Text: Nicholas, Probability: 0.9999776620669459
Text: Taleb, Probability: 0.9999245098504329
Text: RANDOM, Probability: 0.9976557258959531
Text: HOUSE, Probability: 0.9987140698819087
--------------------
Text: 4, Probability: 0.6901684203249978
Text: 1, Probability: 0.31540510401329414
Text: {, Probability: 0.452091057570275
Text: UISSEN, Probability: 0.8856414986931993
Text: f, Probability: 0.4787417144996233
Text: qP[BL, Probability: 0.1671188902967227
Te

## CRAFT

In [29]:
!uv run python ../models/CRAFT-pytorch/test.py --trained_model=../models/CRAFT-pytorch/craft_mlt_25k.pth --test_folder=crops/ --cuda=False

Loading weights from checkpoint (../models/CRAFT-pytorch/craft_mlt_25k.pth)
Test image 1/14: crops/bbox_0.jpg
Test image 2/14: crops/bbox_1.jpg
Test image 3/14: crops/bbox_10.jpg
Test image 4/14: crops/bbox_11.jpg
Test image 5/14: crops/bbox_12.jpg
Test image 6/14: crops/bbox_13.jpg
Test image 7/14: crops/bbox_2.jpg
Test image 8/14: crops/bbox_3.jpg
Test image 9/14: crops/bbox_4.jpg
Test image 10/14: crops/bbox_5.jpg
Test image 11/14: crops/bbox_6.jpg
Test image 12/14: crops/bbox_7.jpg
Test image 13/14: crops/bbox_8.jpg
Test image 14/14: crops/bbox_9.jpg
elapsed time : 10.706003904342651s


In [2]:
j=0
for path_file in os.listdir("./result"): 
    if path_file.endswith(".txt"):
        print(path_file)
        with open(os.path.join("./result",path_file),'r') as file:
            img = Image.open(f"./crops/bbox_{j}.jpg")
            for i,line in enumerate(file):
                if len(line) != 1:
                    line = line.split(',')
                    xcorners = [int(line[i]) for i in [0,2,4,6]]
                    ycorners = [int(line[i]) for i in [1,3,5,7]]
                    crop = img.crop((min(xcorners), min(ycorners), max(xcorners), max(ycorners)))
                    crop.save(f"cropped_words/bbox_{j}_{i}.jpg")
        print(f"Done image bbox_{j}.jpg")
        j+=1

res_bbox_0.txt
Done image bbox_0.jpg
res_bbox_1.txt
Done image bbox_1.jpg
res_bbox_10.txt
Done image bbox_2.jpg
res_bbox_11.txt
Done image bbox_3.jpg
res_bbox_12.txt
Done image bbox_4.jpg
res_bbox_13.txt
Done image bbox_5.jpg
res_bbox_2.txt
Done image bbox_6.jpg
res_bbox_3.txt
Done image bbox_7.jpg
res_bbox_4.txt
Done image bbox_8.jpg
res_bbox_5.txt
Done image bbox_9.jpg
res_bbox_6.txt
Done image bbox_10.jpg
res_bbox_7.txt
Done image bbox_11.jpg
res_bbox_8.txt
Done image bbox_12.jpg
res_bbox_9.txt
Done image bbox_13.jpg


In [27]:
!uv run python ../models/deep-text-recognition-benchmark/demo.py \
--image_folder=cropped_words/ \
--Transformation TPS \
--FeatureExtraction ResNet \
--Prediction CTC \
--SequenceModeling BiLSTM\
--saved_model ../models/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-CTC.pth

model input parameters 32 100 20 1 512 256 37 25 TPS ResNet BiLSTM CTC
loading pretrained model from ../models/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-CTC.pth
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
cropped_words/bbox_1_0.jpg	b                        	0.7556
cropped_words/bbox_1_2.jpg	e                        	0.5213
cropped_words/bbox_1_4.jpg	i                        	0.2092
cropped_words/bbox_1_6.jpg	it                       	0.1974
cropped_words/bbox_1_8.jpg	ie                       	0.1081
cropped_words/bbox_1_10.jpg	e                        	0.4024
cropped_words/bbox_1_12.jpg	random                   	0.5596
cropped_words/bbox_1_14.jpg	house                    	0.4635
cropped_words/bbox_4_0.jpg	toe                      	0.0068
cropped_words/bbox_4_2.jpg	oe                       	0.00

c:\Users\gerar\OneDrive\Desktop\bookAnalysis\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


## CRAFT + EASYOCR 

In [55]:
for file in os.listdir("./cropped_words"):
    img = Image.open(os.path.join('./cropped_words',file))
    for angle in [0,90,-90]:
        result = reader.readtext(img, rotation_info=[angle])

        for (bbox, text, prob) in result:
            print(f'Text: {text}, Probability: {prob}')
    print('-'*20)
    if file[5] != '0':
        break


c:\Users\gerar\OneDrive\Desktop\bookAnalysis\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Text: 2, Probability: 0.3066484125313629
Text: THE, Probability: 0.618259847164154
Text: IHI, Probability: 0.8486661257143797
--------------------
Text: 5, Probability: 0.26847747863061855
Text: Taleb, Probability: 0.9999129410321931
Text: 5, Probability: 0.26847747863061855
--------------------
Text: RANDOM, Probability: 0.9473948083511409
Text: RANDOM, Probability: 0.9473948083511409
Text: RANDOM, Probability: 0.9473948083511409
--------------------
Text: HOUSE, Probability: 0.9995231639171349
Text: HOUSE, Probability: 0.9995231639171349
Text: HOUSE, Probability: 0.9995231639171349
--------------------
Text: 1, Probability: 0.18074429275657522
Text: BLACK, Probability: 0.9995835370858547
Text: 1, Probability: 0.18074429275657522
--------------------
Text: {, Probability: 0.09012758243315844
Text: SWAN, Probability: 0.2753426134586334
Text: VFM S, Probability: 0.5964168988339509
--------------------
Text: ;, Probability: 0.195724676231678
Text: assmm, Probability: 0.8564194049111865
T

## TrOCR

In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

c:\Users\gerar\OneDrive\Desktop\myBookShelf\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# load image from the IAM dataset
url = "https://fki.tic.heia-fr.ch/static/img/a01-122-02.jpg"
# image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
# image = Image.open('./crops/bbox_1.jpg').rotate(90, PIL.Image.NEAREST, expand = 1).convert("RGB")
image = Image.open('./cropped_words/bbox_0_.jpg').rotate(90, PIL.Image.NEAREST, expand = 1).convert("RGB")

pixel_values = processor(image, return_tensors="pt").pixel_values
generated_ids = model.generate(pixel_values)

generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

2 2


## CRAFT + TrOCR

In [7]:
cropped_dir = './cropped_words/'
words_book = []
for file in os.listdir(cropped_dir):
    if file.startswith('bbox_0_'):
        image = Image.open(os.path.join(cropped_dir,file))\
                     .rotate(90, Image.NEAREST, expand = 1)\
                     .convert("RGB")

        pixel_values = processor(image, return_tensors="pt").pixel_values
        generated_ids = model.generate(pixel_values)

        generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(generated_text)

        words_book.append(generated_text)
joined_book = ' '.join(words_book)

the
taleb .
2 2
to
black
swain
nassim .
nicholas


In [11]:
joined_book

'the taleb . 2 2 to black swain nassim . nicholas'

## CRAFT using Hezar library

In [ ]:
from CRAFT.main import load_craft_model, detect_text
net, refine_net = load_craft_model('../models/craft_mlt_25k.pth', cuda=False)
bboxes, polys, score_text = detect_text(
    'crops/bbox_0.jpg', 
    net,
    result_folder='./results_test/',
    cuda=False
)



Loading weights from checkpoint (../models/craft_mlt_25k.pth)


array([[[ 164.95932 ,   39.21679 ],
        [ 220.63152 ,   40.80741 ],
        [ 216.38983 ,  189.26659 ],
        [ 160.71765 ,  187.67596 ]],

       [[ 158.27315 ,  218.10336 ],
        [ 215.12453 ,  219.99843 ],
        [ 207.4738  ,  449.5202  ],
        [ 150.62242 ,  447.62512 ]],

       [[ 144.45312 ,  488.25156 ],
        [ 205.12344 ,  488.25156 ],
        [ 205.12344 ,  678.9297  ],
        [ 144.45312 ,  678.9297  ]],

       [[ 145.03746 ,  976.1459  ],
        [ 190.16525 ,  980.44385 ],
        [ 174.31113 , 1146.9124  ],
        [ 129.18332 , 1142.6144  ]],

       [[ 137.29112 , 1153.3113  ],
        [ 184.1614  , 1156.5813  ],
        [ 169.73788 , 1363.3187  ],
        [ 122.867584, 1360.0487  ]],

       [[ 129.32823 , 1369.8379  ],
        [ 174.80476 , 1372.7719  ],
        [ 166.29625 , 1504.6538  ],
        [ 120.819725, 1501.7198  ]],

       [[  54.892178, 1695.8795  ],
        [ 219.56871 , 1695.8795  ],
        [ 219.56871 , 1730.5482  ],
        [  54.89

In [ ]:
for box in bboxes:
    print()

[164.95932  39.21679 220.63152  40.80741 216.38983 189.26659 160.71765
 187.67596]
[158.27315 218.10336 215.12453 219.99843 207.4738  449.5202  150.62242
 447.62512]
[144.45312 488.25156 205.12344 488.25156 205.12344 678.9297  144.45312
 678.9297 ]
[ 145.03746  976.1459   190.16525  980.44385  174.31113 1146.9124
  129.18332 1142.6144 ]
[ 137.29112  1153.3113    184.1614   1156.5813    169.73788  1363.3187
  122.867584 1360.0487  ]
[ 129.32823  1369.8379    174.80476  1372.7719    166.29625  1504.6538
  120.819725 1501.7198  ]
[  54.892178 1695.8795    219.56871  1695.8795    219.56871  1730.5482
   54.892178 1730.5482  ]
[  69.3375  1730.5485   205.12344 1730.5485   205.12344 1765.2172
   69.3375  1765.2172 ]
